In [1]:

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [2]:
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json

In [3]:
#create spark context
sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")
sc.setLogLevel("WARN")

In [4]:
#batch duration 60s
ssc = StreamingContext(sc, 60)

In [5]:
kafkaStream = KafkaUtils.createStream(ssc, 'cdh57-01-node-01.moffatt.me:2181', 'spark-streaming', {'twitter':1})

In [6]:
#parse inbound message to be json
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [7]:
#count the number of tweets
parsed.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

In [8]:
#get user screen name of each tweets
authors_dstream = parsed.map(lambda tweet: tweet['user']['screen_name'])

In [9]:

author_counts = authors_dstream.countByValue()
author_counts.pprint()

In [10]:
#use transform before sort by
author_counts_sorted_dstream = author_counts.transform(\
  (lambda foo:foo\
   .sortBy(lambda x:( -x[1]))))
#   .sortBy(lambda x:(x[0].lower(), -x[1]))\
#  ))

In [11]:
author_counts_sorted_dstream.pprint()

In [12]:
top_five_authors = author_counts_sorted_dstream.transform\
  (lambda rdd:sc.parallelize(rdd.take(5)))
top_five_authors.pprint()

In [13]:
filtered_authors = author_counts.filter(lambda x:\
                                                x[1]>1 \
                                                or \
                                                x[0].lower().startswith('rm'))

In [14]:
filtered_authors.transform\
  (lambda rdd:rdd\
  .sortBy(lambda x:-x[1]))\
  .pprint()

In [15]:
#list the most common words
parsed.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x:-x[1]))\
    .pprint()

In [16]:
ssc.start()
ssc.awaitTermination(timeout=180)

Py4JJavaError: An error occurred while calling o27.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/streaming/dstream.py", line 631, in <lambda>
    self.func = lambda t, rdd: func(t, prev_func(t, rdd))
  File "/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/streaming/dstream.py", line 295, in <lambda>
    func = lambda t, rdd: oldfunc(rdd)
  File "<ipython-input-10-7fa107b4a593>", line 2, in <lambda>
    author_counts_sorted_dstream = author_counts.transform(  (lambda foo:foo   .sortBy(lambda x:( -x[1]))))
  File "/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/rdd.py", line 694, in sortBy
    return self.keyBy(keyfunc).sortByKey(ascending, numPartitions).values()
  File "/usr/local